In [1]:
from pdkit.gait_time_series import GaitTimeSeries
from pdkit.gait_processor import GaitProcessor

In [2]:
ts = GaitTimeSeries().load_data('../tests/data/cloudupdrs_gait.csv')
gp = GaitProcessor()